In [ ]:
#### Project 

In [ ]:
#### Import packages
from bs4 import BeautifulSoup
import urllib3
import re
import time
from collections import defaultdict
import numpy as np
import tqdm

In [ ]:
### Let's define the urlpage
urlpage = 'https://www.premierleague.com/tables'

In [ ]:
### Let's define a function to get the web page and subtract the part of the html codes we need (function n°1)

def get_page(urlpage,element,html_id):
    # Get page in html
    req = urllib3.PoolManager()
    res = req.request('GET', urlpage)
    row_html = BeautifulSoup(res.data, 'html.parser')
    
    # Return elements that matched the html class in a list
    All_leagues = row_html.find_all('main' ,id= "mainContent")
    
    #Split All_championnat to get each championship
    by_league = str(All_leagues).split('<div class="tableCompetitionExplainedContainer"></div>')
    
    #Return elements and class for the current championship (2023)
    Premier_league_23= by_league[0]
    
    return Premier_league_23

In [ ]:
### Let's test the function n°1

Premier_league_23= get_page(urlpage,
                        'main', "mainContent")
print(Premier_league_23)

In [ ]:
#### Let's define a function to get informations on each team (function n°2)

def extract_team_stats(Premier_league_23, team):
    team_real= team.title()
    pattern = r'data-filtered-table-row-name="{}"'.format(team_real)
    match = re.search(pattern, Premier_league_23)
    start= match.start()
    end = Premier_league_23.index('<tr class="expandable"', start)
    team_data = Premier_league_23[start:end]
    
    position = int(re.findall('data-position="(.*?)">', str(team_data))[0])
    data =  re.findall(r'<td.*?>(\d+)<\/td>', team_data)
    match_played =int(data[0].strip('<td>').strip('</td>'))
    points = int(re.search(r'<td class="points">\d+</td>', team_data).group().strip('<td class="points">').strip('</td>'))
    wins = int(data[1].strip('<td>').strip('</td>'))
    loses = int(data[3].strip('<td>').strip('</td>'))
    drawns = int(data[2].strip('<td>').strip('</td>'))
    but_marqués= int(data[4].strip('<td>').strip('</td>'))
    but_encaissés= int(data[5].strip('<td>').strip('</td>'))

    team_stats = {'position': position,'match_played': match_played,'points': points,'victoires': wins,'defaites': loses,'nuls': drawns,'BM': but_marqués,
        'BE':but_encaissés
    }

    return team_stats

In [ ]:
### Let's test the function n°2

extract_team_stats(Premier_league_23, "manchester city")